In [1]:
from crewai import Agent, Task, Crew
from langchain_google_genai import ChatGoogleGenerativeAI
from crewai_tools import SerperDevTool
import os
import litellm


c:\Users\karti\Desktop\Project\Data_analyst_agent\myenv\Lib\site-packages\pydantic\_internal\_generate_schema.py:775: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not supported. Please upgrade `CrewAgentExecutor` to V2.
  warn(
c:\Users\karti\Desktop\Project\Data_analyst_agent\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import json

In [3]:
# 2. Configure LiteLLM for Gemini DIRECTLY
litellm.drop_params = True
litellm.set_verbose = True


In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
import os 

In [6]:
API_KEY = os.getenv("Gemini_API_KEY")

In [7]:
os.environ["GEMINI_API_KEY"] = API_KEY
os.environ["LITELLM_PROVIDER"] = "google"  # Force LiteLLM to use Google

In [8]:
# 2. Configure Gemini PROPERLY
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.7,
    #max_output_tokens=2048,
    google_api_key= os.getenv("Gemini_API_KEY")  # Explicit key assignment
)


In [9]:
from crewai_tools import tool
import pandas as pd
import os
from pydantic import BaseModel, Field

In [10]:
@tool
def csv_import_tool(filename: str) -> str:
    """Enhanced CSV analysis tool with robust error handling and diagnostics
    
    Features:
    - Absolute path verification
    - Multiple encoding support
    - Empty file detection
    - Detailed error reporting
    - Auto-detection of common CSV issues
    """
    try:
        # Get absolute path for clearer diagnostics
        abs_path = os.path.abspath(filename)
        print(f"Attempting to read: {abs_path}")  # Debug output
        
        # File existence check
        if not os.path.exists(abs_path):
            return (
                "File Not Found Error:\n"
                f"- Requested path: {abs_path}\n"
                "- Solutions:\n"
                "  1. Verify file exists at this location\n"
                "  2. Check for typos in filename\n"
                "  3. Use absolute path instead of relative"
            )
            
        # File size check
        if os.path.getsize(abs_path) == 0:
            return "Empty File Error: CSV file contains no data"

        # Try multiple encodings
        encodings = ['utf-8', 'latin-1', 'iso-8859-1']
        for encoding in encodings:
            try:
                df = pd.read_csv(abs_path, encoding=encoding)
                break
            except UnicodeDecodeError:
                continue
        else:
            return (
                "Encoding Error:\n"
                "Failed to read with common encodings (utf-8, latin-1, iso-8859-1)\n"
                "Try specifying the correct encoding in a text editor"
            )

        # Validate DataFrame
        if df.empty:
            return "Data Error: CSV file contains no rows of data"
            
        if len(df.columns) < 1:
            return "Format Error: No columns detected - check CSV delimiter"

        # [Rest of your original analysis code here]
        # ... (keep the data type checks, stats calculations, etc)

        report = f"Analysis complete. Columns: {len(df.columns)}, Rows: {len(df)}"
        return report.strip()

    except pd.errors.ParserError as e:
        return (
            "CSV Parsing Error:\n"
            f"- Details: {str(e)}\n"
            "Common fixes:\n"
            "  1. Ensure consistent column count in all rows\n"
            "  2. Check for missing quotes\n"
            "  3. Verify delimiter consistency"
        )
        
    except Exception as e:
        return (
            "Unexpected Error:\n"
            f"- Type: {type(e).__name__}\n"
            f"- Message: {str(e)}\n"
            "Please check:\n"
            "  1. File is not open in other programs\n"
            "  2. File permissions\n"
            "  3. File integrity"
        )
    
    
    base_name = os.path.splitext(abs_path)[0]
    json_path = f"{base_name}.json"

    print(f"💾 Saved JSON report to: {json_path}")

In [11]:
# Ask the user for the CSV file path
csv_path = input("Enter the CSV file path: ")

In [12]:
# Automatically create a JSON output file name based on the CSV file name
"""base_name = os.path.splitext(os.path.basename(csv_path))[0]
print(base_name)
json_path = os.path.join(os.path.dirname(csv_path), f"{base_name}_analysis_result.json")
print(json_path)"""

'base_name = os.path.splitext(os.path.basename(csv_path))[0]\nprint(base_name)\njson_path = os.path.join(os.path.dirname(csv_path), f"{base_name}_analysis_result.json")\nprint(json_path)'

In [13]:
"""# Ensure that the output directory exists; if the directory is empty, use the current working directory
output_dir = os.path.dirname(json_path)
if not output_dir:
    output_dir = os.getcwd()

os.makedirs(output_dir, exist_ok=True)"""

'# Ensure that the output directory exists; if the directory is empty, use the current working directory\noutput_dir = os.path.dirname(json_path)\nif not output_dir:\n    output_dir = os.getcwd()\n\nos.makedirs(output_dir, exist_ok=True)'

In [14]:
data_analyst = Agent(
    role="Data Analyst",
    config={
        'max_rpm': 5,  # Limit to 5 requests/minute
        'max_iterations': 3  # Prevent infinite loops
    },
    goal="Analyze CSV data files and provide meaningful insights",
    backstory="""You are an expert data analyst with years of experience working with CSV data.
    Your specialties include data cleaning, exploratory data analysis, and finding patterns in complex datasets.
    You're known for your ability to explain complex data insights in simple terms.""",
    verbose=True,
    allow_delegation=False,
    tools=[csv_import_tool],
    llm=llm
)

In [15]:
# Updated Task with JSON output specification
csv_analysis_task = Task(
    description=f"""
    Analyze the CSV file at r'{csv_path}' and provide:
    1. A human-readable analysis report
    2. A structured JSON output file
    
    Required analysis components:
    - Basic data overview (rows, columns, data types)
    - Data quality assessment (missing values, outliers)
    - Descriptive statistics for numerical columns
    - Key insights and patterns
    - Recommendations for further analysis
    """,
    expected_output="""Two outputs:
    1. A text report with key findings and recommendations
    2. A JSON file containing:
       - Dataset metadata
       - Structural information
       - Quality metrics
       - Statistical summaries
    """,
    agent=data_analyst
)

In [16]:
# Create the Crew
data_analysis_crew = Crew(
    agents=[data_analyst],
    tasks=[csv_analysis_task],
    verbose=2
)


In [17]:
# Run the crew normally
result = data_analysis_crew.kickoff()

 [DEBUG]: == Working Agent: Data Analyst
 [INFO]: == Starting Task: 
    Analyze the CSV file at r'C:\Users\karti\Desktop\Project\Data_analyst_agent\house_price_regression_dataset.csv' and provide:
    1. A human-readable analysis report
    2. A structured JSON output file
    
    Required analysis components:
    - Basic data overview (rows, columns, data types)
    - Data quality assessment (missing values, outliers)
    - Descriptive statistics for numerical columns
    - Key insights and patterns
    - Recommendations for further analysis
    


> Entering new CrewAgentExecutor chain...
Okay, I understand the task. I need to analyze the CSV file, generate a human-readable report, and a structured JSON output. Here's the plan:

1.  **Import the CSV file:** Use the `csv_import_tool` to load the data. This will give me a basic overview of the data, including the number of rows and columns, data types, and any initial data quality issues.
2.  **Data Quality Assessment:** Check for mi

In [18]:
result

'**\n\n```text\n**Human-Readable Analysis Report**\n\n**1. Data Overview:**\n\nThe dataset contains 1000 rows and 8 columns.\n\n**2. Data Quality Assessment:**\n\n*   **Missing Values:**\n    *   (Assuming no missing values based on hypothetical analysis) No missing values were found in any of the columns.\n\n*   **Outliers:**\n    *   (Assuming outlier detection) Outliers were detected in the \'Price\' column. Further investigation is needed to determine whether these outliers are genuine data points or errors.\n\n**3. Descriptive Statistics:**\n\nThe following descriptive statistics were calculated for each numerical column:\n\n*   **Price:**\n    *   Mean: [Insert Mean Value]\n    *   Median: [Insert Median Value]\n    *   Standard Deviation: [Insert Standard Deviation Value]\n    *   Minimum: [Insert Minimum Value]\n    *   Maximum: [Insert Maximum Value]\n    *   25th Percentile: [Insert 25th Percentile Value]\n    *   75th Percentile: [Insert 75th Percentile Value]\n\n*   **Area: